# Web Scraping de casas en venta en la plataforma Lamudi

El siguiente es un script de muestra para hacer web scraping en una plataforma de vienes raices 
con la libreria selenium de python. La finalidad es extraer datos de los diferentes inmuebles 
en venta y almacenarlos en un archivo csv para su posterior análisis


## Importar librerias

In [111]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import numpy as np

## Número de páginas

Definimos la variable `num_paginas`, la cual le indica al script el número de páginas 
de las cuales queremos extraer información. Cada página contiene información de 30 inmuebles.

In [112]:
num_paginas = 10

## Navegador

El navegador web utilizado para el scraping fue `Edge` el cual viene preinstalado en `Windows 10`

In [113]:
driver = webdriver.Edge()

## Función para extraer urls de cada inmueble

In [114]:
def extraer_urls(num_paginas = 1):
    urls = []
    for i in range(1,num_paginas+1):
        driver.get(f"https://www.lamudi.com.mx/casa/for-sale/?page={i}")
        driver.implicitly_wait(3)
        container = driver.find_element(By.XPATH, "/html/body/div[4]/div[2]/div[2]/div[2]")
        inmuebles = container.find_elements(By.CLASS_NAME,"ListingCell-agent-redesign")
        
        for inmueble in inmuebles:
            url = inmueble.find_element(By.CLASS_NAME,"ListingCell-MainImage").find_element(By.CSS_SELECTOR,"a").get_attribute("href")
            urls.append(url)
    return urls 

## Función para extraer la informacion de cada inmueble

In [115]:
def extraer_info_inmueble(url):
    try:
        driver.get(url)
        driver.implicitly_wait(3)
        container = driver.find_element(By.CLASS_NAME,"PdpV5-content-mainColumn")
        
    except NoSuchElementException:
        return None
        
    info = {
        'Título':None,
        'Descripción':None,
        'Precio':None,
        'Recámaras':None,
        'Baños':None,
        'Estacionamientos':None,
        'Construidos':None,
        'Terreno':None,
        'Url':url
    }

    # Extraer titilo
    info['Título'] = container.find_element(By.CLASS_NAME,"Title-pdp-title-wrapper").find_element(By.CSS_SELECTOR,"h1").text

    # Extraer descripción
    info['Descripción'] = container.find_element(By.CLASS_NAME,"ViewMore-text-description").text

    # Extraer precio
    info['Precio'] = container.find_element(By.CLASS_NAME,"Title-pdp-price").find_element(By.CSS_SELECTOR,"span").text

    # Extraer info de recamaras, baños, estacionamientos, m^2 construidos, m^2 terreno
    for i in range(1,3):
        j=1
        while 1:
            try:
                categoria = container.find_element(By.XPATH,f"//section[1]/div[2]/div[1]/div[{i}]/div[{j}]").find_element(By.CLASS_NAME,"ellipsis").text
                numero = container.find_element(By.XPATH,f"//section[1]/div[2]/div[1]/div[{i}]/div[{j}]").find_element(By.CLASS_NAME,"last").text
                j+=1
            
                if categoria.find('Recámaras') != -1:
                    info['Recámaras'] = numero
                if categoria.find('Baños') != -1:
                    info['Baños'] = numero
                if categoria.find('Estacionamientos') != -1:
                    info['Estacionamientos'] = numero
                if categoria.find('Construidos') != -1:
                    info['Construidos'] = numero
                if categoria.find('Terreno') != -1:
                    info['Terreno'] = numero
                
            except NoSuchElementException:
                break
        
    return info

## Extraer data

Extraemos los datos y los almacenamos en un Data Frame de pandas

In [116]:
def extraer_data():
    data = {
        'Título':[],
        'Descripción':[],
        'Precio':[],
        'Recámaras':[],
        'Baños':[],
        'Estacionamientos':[],
        'Construidos':[],
        'Terreno':[],
        'Url':[]
    }

    urls = extraer_urls(num_paginas)

    for i,url in enumerate(urls):
        info = extraer_info_inmueble(url)
        if info != None:
            for key in data.keys():
                data[key].append(info[key])
        
        # Imprimir barra de progreso
        print(f"|{'█'*int((i*40)/(len(urls)-1))}{' '*(40 - int((i*40)/(len(urls)-1)))}| {int(((i*40)/(len(urls)-1))*2.5)}%",end='\r')  
        
    return data

data = extraer_data()

In [117]:
df = pd.DataFrame(data)
df.head(5)

,Título,Descripción,Precio,Recámaras,Baños,Estacionamientos,Construidos,Terreno,Url
0,Casa en Senderos de Monteverde,"Moderna casa con 4 recámaras y 4 baños, todas ...","$ 3,450,000",4,4,3,160,143,https://www.lamudi.com.mx/casa-en-senderos-de-...
1,Casa de Autor en Venta Zibatá,Wiggot: pSBi4jM\n\n\nHermosa casa con espacios...,"$ 3,995,000",3,4,2,184,148,https://www.lamudi.com.mx/casa-de-autor-en-ven...
2,Casa en venta Riviera Veracruzana acceso a la ...,Casa de 1 Nivel con 3 Recamaras en la Riviera...,"$ 3,050,000",3,2.5,2,195,159,https://www.lamudi.com.mx/casa-en-venta-con-ac...
3,estilo único- también tenemos preventa- amplio...,PLANTA BAJA:\n* Cochera para 4 autos (2 techad...,"$ 9,750,000",5,5,None,500,650,https://www.lamudi.com.mx/estilo-nico-tambi-n-...
4,Casa Malaga Residencial!!,Ven y conoce uno de nuestros mejores prototipo...,"$ 3,100,000",3,2.5,2,136,144,https://www.lamudi.com.mx/casa-malaga-residenc...


In [118]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297 entries, 0 to 296
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Título            297 non-null    object
 1   Descripción       297 non-null    object
 2   Precio            297 non-null    object
 3   Recámaras         295 non-null    object
 4   Baños             294 non-null    object
 5   Estacionamientos  278 non-null    object
 6   Construidos       295 non-null    object
 7   Terreno           291 non-null    object
 8   Url               297 non-null    object
dtypes: object(9)
memory usage: 21.0+ KB


## Modificamos los tipos de datos en el data frame

In [123]:
#df['Precio'] = df['Precio'].apply(lambda x: int(x[2:].replace(',','')) if x.find('$') != -1 else np.nan)
df['Recámaras'] = df['Recámaras'].apply(lambda x: int(x) if x != None else np.nan)
df['Baños'] = df['Baños'].apply(lambda x: float(x) if x != None else np.nan)
df['Estacionamientos'] = df['Estacionamientos'].apply(lambda x: int(x) if x != None else np.nan)
df['Construidos'] = df['Construidos'].apply(lambda x: float(x) if x != None else np.nan)
df['Terreno'] = df['Terreno'].apply(lambda x: float(x) if x != None else np.nan)

In [124]:
df.rename(columns={'Construidos':'Construidos m2', 'Terreno':'Terreno m2'}, inplace=True)

In [125]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297 entries, 0 to 296
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Título            297 non-null    object 
 1   Descripción       297 non-null    object 
 2   Precio            297 non-null    object 
 3   Recámaras         295 non-null    float64
 4   Baños             294 non-null    float64
 5   Estacionamientos  278 non-null    float64
 6   Construidos m2    295 non-null    float64
 7   Terreno m2        291 non-null    float64
 8   Url               297 non-null    object 
dtypes: float64(5), object(4)
memory usage: 21.0+ KB


## Guardaos los datos en formatos csv y excel

In [126]:
df.to_csv('Data/casas-en-venta.csv', index=False)

In [127]:
df.to_excel('Data/casas-en-venta.xlsx', index=False)